In [1]:
import pandas as pd
import datetime as dt
from datetime import date


In [2]:
today = date.today()
today_str = today.strftime("%Y%m%d")
print(today_str)


20190507


In [3]:
df = pd.read_hdf("X:/Applications/Admissions/funnel/data/stage_data", key="weekly")
df = df[(df["year_term"] > "2011.Spring")]


In [4]:
summ = df.groupby(["year_term", "stage"]).sum()
summ_t = summ.transpose()


In [5]:
print(summ.shape)
summ.head()


(48, 54)


00  01  02  03  04  05  06  07  08  09  ...   44   45  \
year_term   stage                                              ...             
2012.Fall   Accepted    1   1   1   1   1   1   1   2   2   2  ...  416  413   
            Applied     1   1   1   2   5   6   7  11  14  25  ...  585  583   
            Deposited   3   3   3   3   3   3   3   4   4   4  ...  368  368   
2012.Spring Accepted    0   0   0   0   0   0   0   0   0   1  ...    1    1   
            Applied     0   0   0   0   1   1   1   1   1   1  ...    2    2   

                        46   47   48   49   50   51   52   53  
year_term   stage                                              
2012.Fall   Accepted   396  395  359  360  343  290  276  276  
            Applied    562  557  509  510  493  436  423  423  
            Deposited  369  369  372  372  373  365  365  365  
2012.Spring Accepted     1    1    1    1    1    1    1    1  
            Applied      2    2    2    2    2    2    2    2  

[5 rows x 54 columns]

In [6]:
summ_t.head()


year_term 2012.Fall                   2012.Spring                   2013.Fall  \
stage      Accepted Applied Deposited    Accepted Applied Deposited  Accepted   
00                1       1         3           0       0         0        18   
01                1       1         3           0       0         0        20   
02                1       1         3           0       0         0        23   
03                1       2         3           0       0         0        24   
04                1       5         3           0       1         0        27   

year_term                   2013.Spring  ... 2018.Fall 2018.Spring          \
stage     Applied Deposited    Accepted  ... Deposited    Accepted Applied   
00             83         9           8  ...         7           2       4   
01             73         9           8  ...         6           2       4   
02             77        10           8  ...         6           2       5   
03             93        13          13  ...         6           2       6   
04            105        13          14  ...         6           2       8   

year_term           2019.Fall                   2019.Spring                    
stage     Deposited  Accepted Applied Deposited    Accepted Applied Deposited  
00                3         2      19         1           2       8         2  
01                3         2      29         1           4      11         2  
02                3         2      37         1           5      13         3  
03                3         2      46         1           5      14         3  
04                3         2      56         1           6      14         3  

[5 rows x 48 columns]

In [7]:
b_yr = 2012
e_yr = 2019
term = 'Fall'

terms = [f"{y}.{term}" for y in range(b_yr, e_yr+1)]
stages = [ 'Applied', 'Accepted', 'Deposited']
for st in stages:
    df_sumt = summ_t.loc[:, (terms, st)]


In [8]:
print(today_str)
writer = pd.ExcelWriter(
    f"Historical{term}Enrollment_{b_yr:d}-{e_yr:d}_{today_str}.xlsx"
)
df_sumt.T.reset_index().T.to_excel(writer, f"old", index=False)
writer.save()


20190507


In [ ]:
summ.loc[(summ["year_term"] == "2019.Fall") & (summ["stage"] == "Deposited")]


In [7]:
def adm_week(d):
    """
    returns calendar week number and Admissions Week Number for a given date, d
    """
    year = d.year
    if d >= date(year, 9, 1):
        adm_year_start = year
    else:
        adm_year_start = year - 1

    week_number = d.isocalendar()[1]
    adm_start_week = date(adm_year_start, 9, 1).isocalendar()[1]

    if week_number >= adm_start_week:
        adm_week_number = week_number - adm_start_week
    else:
        adm_week_number = 53 + (week_number - adm_start_week)

    return (week_number, adm_week_number)


week_number, adm_week_number = adm_week(today)


In [8]:
def adm_week_end_date(awn, acad_yr):
    """
    returns calendar date for the Friday of the given Admissions Week Number, awn
        and academic year, acad_yr
    """
    
    adm_start_date = dt.date((acad_yr-1), 9, 1)
    adm_start_week = adm_start_date.isocalendar()[1]
    
    if awn < (53 - adm_start_week):
        wn = adm_start_week + awn
        year = acad_yr - 1
    else:
        wn = awn - (53 - adm_start_week)
        year = acad_yr

    week_end = dt.datetime.strptime(f'{year}-{wn}-1', "%Y-%W-%w").date() + dt.timedelta(days=6.9)
    
    return(week_end)


In [9]:
print(adm_week_end_date(0, 2019))
print(adm_week_end_date(16, 2019))
print(adm_week_end_date(17, 2019))
print(adm_week_end_date(18, 2019))
print(adm_week_end_date(19, 2019))
print(adm_week_end_date(52, 2019))
print(adm_week_end_date(53, 2019))

2018-09-02
2018-12-23
2018-12-30
2019-01-06
2019-01-13
2019-09-01
2019-09-08


In [8]:
week = adm_week(today)[1]


In [9]:
print("2019.Fall", str(week), summ.loc[("2019.Fall", "Deposited")][str(week)])
print("2018.Fall", str(week), summ.loc[("2018.Fall", "Deposited")][str(week)])
print("2017.Fall", str(week), summ.loc[("2017.Fall", "Deposited")][str(week)])
print("2016.Fall", str(week), summ.loc[("2016.Fall", "Deposited")][str(week)])
print("2015.Fall", str(week), summ.loc[("2015.Fall", "Deposited")][str(week)])
print("2014.Fall", str(week), summ.loc[("2014.Fall", "Deposited")][str(week)])

summ.loc[("2019.Fall", "Deposited")][str(week)] - summ.loc[("2018.Fall", "Deposited")][
    str(week)
]


2019.Fall 36 166
2018.Fall 36 182
2017.Fall 36 191
2016.Fall 36 249
2015.Fall 36 254
2014.Fall 36 213


-16

In [10]:
day = date(2019, 5, 1)
print(day)
week = adm_week(day)[1]
print(week)

print("2019.Fall\t", str(week), "\t", summ.loc[("2019.Fall", "Deposited")][str(week)])
print("2018.Fall\t", str(week), "\t", summ.loc[("2018.Fall", "Deposited")][str(week)])
print("2017.Fall\t", str(week), "\t", summ.loc[("2017.Fall", "Deposited")][str(week)])
print("2016.Fall\t", str(week), "\t", summ.loc[("2016.Fall", "Deposited")][str(week)])
print("2015.Fall\t", str(week), "\t", summ.loc[("2015.Fall", "Deposited")][str(week)])
print("2014.Fall\t", str(week), "\t", summ.loc[("2014.Fall", "Deposited")][str(week)])

print(summ.loc[("2019.Fall", "Deposited")][str(week)])
print(summ.loc[("2018.Fall", "Deposited")][str(week)])
print(summ.loc[("2017.Fall", "Deposited")][str(week)])
print(summ.loc[("2016.Fall", "Deposited")][str(week)])
print(summ.loc[("2015.Fall", "Deposited")][str(week)])
print(summ.loc[("2014.Fall", "Deposited")][str(week)])


2019-05-01
36
2019.Fall	 36 	 166
2018.Fall	 36 	 182
2017.Fall	 36 	 191
2016.Fall	 36 	 249
2015.Fall	 36 	 254
2014.Fall	 36 	 213
166
182
191
249
254
213


In [11]:
day = date(2019, 8, 25)
print(day)
week = adm_week(day)[1]
print(week)

print("2019.Fall\t", str(week), "\t", summ.loc[("2019.Fall", "Deposited")][str(week)])
print("2018.Fall\t", str(week), "\t", summ.loc[("2018.Fall", "Deposited")][str(week)])
print("2017.Fall\t", str(week), "\t", summ.loc[("2017.Fall", "Deposited")][str(week)])
print("2016.Fall\t", str(week), "\t", summ.loc[("2016.Fall", "Deposited")][str(week)])
print("2015.Fall\t", str(week), "\t", summ.loc[("2015.Fall", "Deposited")][str(week)])
print("2014.Fall\t", str(week), "\t", summ.loc[("2014.Fall", "Deposited")][str(week)])

print(summ.loc[("2019.Fall", "Deposited")][str(week)])
print(summ.loc[("2018.Fall", "Deposited")][str(week)])
print(summ.loc[("2017.Fall", "Deposited")][str(week)])
print(summ.loc[("2016.Fall", "Deposited")][str(week)])
print(summ.loc[("2015.Fall", "Deposited")][str(week)])
print(summ.loc[("2014.Fall", "Deposited")][str(week)])


2019-08-25
52
2019.Fall	 52 	 166
2018.Fall	 52 	 245
2017.Fall	 52 	 260
2016.Fall	 52 	 286
2015.Fall	 52 	 319
2014.Fall	 52 	 304
166
245
260
286
319
304


In [12]:
day = date(2019, 9, 1)
print(day)
week = 53
print(week)

print("2019.Fall\t", str(week), "\t", summ.loc[("2019.Fall", "Deposited")][str(week)])
print("2018.Fall\t", str(week), "\t", summ.loc[("2018.Fall", "Deposited")][str(week)])
print("2017.Fall\t", str(week), "\t", summ.loc[("2017.Fall", "Deposited")][str(week)])
print("2016.Fall\t", str(week), "\t", summ.loc[("2016.Fall", "Deposited")][str(week)])
print("2015.Fall\t", str(week), "\t", summ.loc[("2015.Fall", "Deposited")][str(week)])
print("2014.Fall\t", str(week), "\t", summ.loc[("2014.Fall", "Deposited")][str(week)])

print(summ.loc[("2019.Fall", "Deposited")][str(week)])
print(summ.loc[("2018.Fall", "Deposited")][str(week)])
print(summ.loc[("2017.Fall", "Deposited")][str(week)])
print(summ.loc[("2016.Fall", "Deposited")][str(week)])
print(summ.loc[("2015.Fall", "Deposited")][str(week)])
print(summ.loc[("2014.Fall", "Deposited")][str(week)])


2019-09-01
53
2019.Fall	 53 	 166
2018.Fall	 53 	 245
2017.Fall	 53 	 260
2016.Fall	 53 	 286
2015.Fall	 53 	 319
2014.Fall	 53 	 304
166
245
260
286
319
304


In [10]:
print("2017.Fall", "53", summ.loc[("2017.Fall", "Deposited")]["53"])
print("2017.Fall", str(week), summ.loc[("2017.Fall", "Deposited")][str(week)])

summ.loc[("2017.Fall", "Deposited")]["53"] - summ.loc[("2017.Fall", "Deposited")][
    str(week)
]


2017.Fall 53 259
2017.Fall 51 257


2

In [11]:
weeks = range(0, 54)
for w in weeks:
    f = f"{w:02d}"
    yet_to_deposit = (
        summ.loc[("2017.Fall", "Deposited")]["53"]
        - summ.loc[("2017.Fall", "Deposited")][f]
    )
    print(w, yet_to_deposit)


0 249
1 249
2 248
3 248
4 248
5 248
6 248
7 248
8 248
9 248
10 246
11 246
12 246
13 243
14 238
15 235
16 228
17 227
18 227
19 220
20 216
21 212
22 209
23 203
24 202
25 196
26 181
27 175
28 173
29 167
30 159
31 148
32 140
33 132
34 113
35 88
36 69
37 62
38 58
39 48
40 40
41 34
42 31
43 25
44 18
45 14
46 10
47 9
48 -2
49 -9
50 2
51 2
52 0
53 0


In [12]:
summ.describe()


,00,01,02,03,04,05,06,07,08,09,...,44,45,46,47,48,49,50,51,52,53
count,42.000000,42.000000,42.000000,42.000000,42.000000,42.000000,42.000000,42.000000,42.000000,42.000000,...,42.000000,42.000000,42.000000,42.000000,42.000000,42.000000,42.000000,42.000000,42.000000,42.000000
mean,17.928571,18.476190,20.404762,23.095238,25.500000,28.880952,33.619048,38.928571,47.238095,56.976190,...,289.857143,287.333333,285.595238,284.333333,278.047619,266.690476,255.571429,249.476190,246.619048,246.619048
std,23.324218,23.519829,26.454802,30.240346,33.319774,38.120606,45.396869,53.941428,64.639643,79.063879,...,326.758998,324.003588,322.986752,321.858980,316.542137,302.700970,292.897730,290.406967,288.641727,288.641727
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
25%,3.000000,3.000000,3.000000,3.000000,3.250000,3.250000,4.250000,5.250000,5.250000,6.250000,...,28.250000,28.000000,28.000000,28.000000,27.250000,27.250000,27.250000,27.250000,27.250000,27.250000
50%,8.500000,9.000000,9.500000,12.000000,12.000000,12.000000,14.500000,16.500000,17.500000,19.500000,...,138.000000,140.000000,142.500000,144.000000,144.000000,145.500000,148.000000,148.000000,148.000000,148.000000
75%,23.000000,23.750000,25.500000,30.000000,31.500000,38.750000,41.500000,50.750000,70.250000,77.500000,...,435.500000,419.000000,409.500000,396.000000,372.250000,354.250000,344.250000,332.500000,324.250000,324.250000
max,90.000000,89.000000,103.000000,124.000000,132.000000,146.000000,174.000000,210.000000,245.000000,296.000000,...,1024.000000,992.000000,984.000000,974.000000,941.000000,939.000000,938.000000,938.000000,938.000000,938.000000


In [13]:
summ.index


MultiIndex(levels=[['2012.Fall', '2012.Spring', '2013.Fall', '2013.Spring', '2014.Fall', '2014.Spring', '2015.Fall', '2015.Spring', '2016.Fall', '2016.Spring', '2017.Fall', '2017.Spring', '2018.Fall', '2018.Spring'], ['Accepted', 'Applied', 'Deposited']],
           labels=[[0, 0, 0, 1, 1, 1, 2, 2, 2, 3, 3, 3, 4, 4, 4, 5, 5, 5, 6, 6, 6, 7, 7, 7, 8, 8, 8, 9, 9, 9, 10, 10, 10, 11, 11, 11, 12, 12, 12, 13, 13, 13], [0, 1, 2, 0, 1, 2, 0, 1, 2, 0, 1, 2, 0, 1, 2, 0, 1, 2, 0, 1, 2, 0, 1, 2, 0, 1, 2, 0, 1, 2, 0, 1, 2, 0, 1, 2, 0, 1, 2, 0, 1, 2]],
           names=['year_term', 'stage'])

In [14]:
# Goal
GOAL = 260
summ.loc[("2018.Fall", "Deposited")]["53"] = GOAL


In [15]:
df1 = summ.copy()
weeks = range(0, 54)
for w in weeks:
    f = f"{w:02d}"
    df1[f] = summ.loc[:]["53"] - summ.loc[:][f]

print(df1.shape)


(42, 54)


In [16]:
df1.head()


00   01   02   03   04   05   06   07   08   09 ...  \
year_term   stage                                                       ...   
2012.Fall   Accepted   261  261  261  261  261  261  261  260  260  260 ...   
            Applied    391  391  391  390  387  386  386  383  381  371 ...   
            Deposited  360  360  360  360  360  360  360  359  359  359 ...   
2012.Spring Accepted     1    1    1    1    1    1    1    1    1    0 ...   
            Applied      2    2    2    2    1    1    1    1    1    1 ...   

                        44   45   46   47  48  49  50  51  52  53  
year_term   stage                                                  
2012.Fall   Accepted  -140 -137 -120 -119 -83 -84 -67 -14   0   0  
            Applied   -162 -160 -139 -134 -86 -87 -70 -13   0   0  
            Deposited   -3   -3   -4   -4  -7  -7  -8   0   0   0  
2012.Spring Accepted     0    0    0    0   0   0   0   0   0   0  
            Applied      0    0    0    0   0   0   0   0   0   0  

[5 rows x 54 columns]

In [17]:
df1_t = df1.transpose()


In [18]:
df1_t.head()


year_term 2012.Fall                   2012.Spring                   2013.Fall  \
stage      Accepted Applied Deposited    Accepted Applied Deposited  Accepted   
00              261     391       360           1       2        39       279   
01              261     391       360           1       2        39       277   
02              261     391       360           1       2        39       274   
03              261     390       360           1       2        39       273   
04              261     387       360           1       1        39       270   

year_term                   2013.Spring    ...    2017.Fall 2017.Spring  \
stage     Applied Deposited    Accepted    ...    Deposited    Accepted   
00            229       290          21    ...          249           8   
01            239       290          21    ...          249           8   
02            235       289          21    ...          248           8   
03            219       286          16    ...          248           8   
04            207       286          15    ...          248           6   

year_term                   2018.Fall                   2018.Spring          \
stage     Applied Deposited  Accepted Applied Deposited    Accepted Applied   
00              6        10       908     911       257          21      23   
01              6        10       907     905       257          21      23   
02              6        10       907     903       257          21      22   
03              5        10       907     895       257          21      21   
04              4        10       907     887       257          21      19   

year_term            
stage     Deposited  
00               18  
01               18  
02               18  
03               18  
04               18  

[5 rows x 42 columns]

In [19]:
# from bokeh.layouts import widgetbox, row
# from bokeh.models.widgets import MultiSelect, RadioGroup, Select
# from bokeh.plotting import figure, curdoc
from bokeh.plotting import figure, output_file, show

# from bokeh.palettes import Blues9
from bokeh.palettes import Set2_8
from bokeh.io import output_notebook


In [20]:
output_notebook()
output_file("DepositsToEndOFYear.html")


Loading BokehJS ...

In [21]:
TOOLS = "pan,wheel_zoom,box_zoom,save,reset"


In [22]:
stage = "Deposited"
term = "2018.Fall"

all_terms = sorted(list(df["year_term"].dropna().unique()))
all_terms = [l for l in all_terms if "Fall" in l]
all_terms.remove(term)
# all_terms.remove('2013.Fall')
all_terms.remove("2012.Fall")
all_terms.reverse()
y_max = df1_t[(term, stage)].max()
y_min = df1_t[(term, stage)].min()
for t in all_terms:
    ymx = df1_t[(t, stage)].max()
    ymn = df1_t[(t, stage)].min()
    if ymx > y_max:
        y_max = ymx
    if ymn < y_min:
        y_min = ymn

print(term, stage, y_min, y_max)


2018.Fall Deposited -15 317


In [23]:
print(all_terms)


['2017.Fall', '2016.Fall', '2015.Fall', '2014.Fall', '2013.Fall']


In [24]:
title = f"'{stage}' Gained to End of Year,  Goal={GOAL}  (Week {adm_week_number:d})"


In [25]:
p = figure(
    plot_width=800,
    plot_height=600,
    title=title,
    x_axis_label="Admissions Week Number (year starts Sept 1)",
    y_axis_label=stage,
    tools=TOOLS,
    x_range=(0, 54),
    y_range=(y_min * 1.05, y_max * 1.05),
)

c = 0
for t in all_terms:
    # p.line(df1_t.index, df1_t[(t, stage)], color=Blues9[c], legend=t)
    p.line(df1_t.index, df1_t[(t, stage)], color=Set2_8[c], legend=t)
    if c <= 7:
        c += 1

p.line(df1_t.index, df1_t[(term, stage)], color="red", line_width=2, legend=term)

# week_number line
p.line(
    (adm_week_number, adm_week_number),
    (-1000, 5000),
    color="green",
    line_width=0.8,
    line_dash="dashed",
    legend=f"Week {adm_week_number:d}",
    alpha=0.8,
)

p.legend.location = "top_right"

show(p)


In [26]:
# get list of 2018.Fall applicants
dep = (
    df.loc[
        (df["year_term"] == "2018.Fall") & (df["stage"] == "Deposited") & (df["53"] > 0)
    ]
    .sort_values(["PEOPLE_CODE_ID"])
    .drop_duplicates(["PEOPLE_CODE_ID"])
)
print(dep.shape)


(249, 58)


In [27]:
dep.head()


,year_term,stage,PEOPLE_CODE_ID,00,01,02,03,04,05,06,...,45,46,47,48,49,50,51,52,53,curriculum
33973,2018.Fall,Deposited,P000026021,0,0,0,0,0,0,0,...,1,1,1,1,1,1,1,1,1,FEFM
33980,2018.Fall,Deposited,P000045865,0,0,0,0,0,0,0,...,1,1,1,1,1,1,1,1,1,NRCM
33981,2018.Fall,Deposited,P000045910,0,0,0,0,0,0,0,...,0,0,0,0,0,1,1,1,1,FEFM
33982,2018.Fall,Deposited,P000047104,0,0,0,0,0,0,0,...,1,1,1,1,1,1,1,1,1,MNGTS
33983,2018.Fall,Deposited,P000047242,0,0,0,0,0,0,0,...,1,1,1,1,1,1,1,1,1,PSYCH


In [28]:
dep.to_csv("2018-Fall-Deposited.csv")


In [ ]:
df.loc[(df["PEOPLE_CODE_ID"] == "P000054911")]


In [30]:
# melt
import numpy as np

a = [
    summ.loc[("2017.Fall", "Deposited")]["53"]
    - summ.loc[("2017.Fall", "Deposited")][str(week)],
    summ.loc[("2016.Fall", "Deposited")]["53"]
    - summ.loc[("2016.Fall", "Deposited")][str(week)],
    summ.loc[("2015.Fall", "Deposited")]["53"]
    - summ.loc[("2015.Fall", "Deposited")][str(week)],
    summ.loc[("2014.Fall", "Deposited")]["53"]
    - summ.loc[("2014.Fall", "Deposited")][str(week)],
    summ.loc[("2013.Fall", "Deposited")]["53"]
    - summ.loc[("2013.Fall", "Deposited")][str(week)],
    summ.loc[("2012.Fall", "Deposited")]["53"]
    - summ.loc[("2012.Fall", "Deposited")][str(week)],
]
print(a)
print(sum(a))
print(sum(a) / 6)
print(np.mean(a))


[2, -4, -6, 0, -2, 0]
-10
-1.6666666666666667
-1.6666666666666667


In [22]:
base = dt.date(2018, 12, 20)
#date_list = [str(base + dt.timedelta(days=x)) for x in range(0, 21)]
date_list = [base + dt.timedelta(days=x) for x in range(0, 21)]
print(date_list)
for d in date_list:
    wn, awn = adm_week(d)
    end = adm_week_end_date(awn, 2019) 
    print(d, end, awn, wn)

[datetime.date(2018, 12, 20), datetime.date(2018, 12, 21), datetime.date(2018, 12, 22), datetime.date(2018, 12, 23), datetime.date(2018, 12, 24), datetime.date(2018, 12, 25), datetime.date(2018, 12, 26), datetime.date(2018, 12, 27), datetime.date(2018, 12, 28), datetime.date(2018, 12, 29), datetime.date(2018, 12, 30), datetime.date(2018, 12, 31), datetime.date(2019, 1, 1), datetime.date(2019, 1, 2), datetime.date(2019, 1, 3), datetime.date(2019, 1, 4), datetime.date(2019, 1, 5), datetime.date(2019, 1, 6), datetime.date(2019, 1, 7), datetime.date(2019, 1, 8), datetime.date(2019, 1, 9)]
2018-12-20 2018-12-23 16 51
2018-12-21 2018-12-23 16 51
2018-12-22 2018-12-23 16 51
2018-12-23 2018-12-23 16 51
2018-12-24 2018-12-30 17 52
2018-12-25 2018-12-30 17 52
2018-12-26 2018-12-30 17 52
2018-12-27 2018-12-30 17 52
2018-12-28 2018-12-30 17 52
2018-12-29 2018-12-30 17 52
2018-12-30 2018-12-30 17 52
2018-12-31 2019-01-13 19 1
2019-01-01 2019-01-13 19 1
2019-01-02 2019-01-13 19 1
2019-01-03 2019-01-

In [ ]:
print(today.year)
for w in range(0,54):
    end = adm_week_end_date(w, 2019) 
    print(w, end, adm_week(end)[1])

In [14]:
summ_t.head()

year_term 2012.Fall                   2012.Spring                   2013.Fall  \
stage      Accepted Applied Deposited    Accepted Applied Deposited  Accepted   
00                1       1         3           0       0         0        18   
01                1       1         3           0       0         0        20   
02                1       1         3           0       0         0        23   
03                1       2         3           0       0         0        24   
04                1       5         3           0       1         0        27   

year_term                   2013.Spring  ... 2018.Fall 2018.Spring          \
stage     Applied Deposited    Accepted  ... Deposited    Accepted Applied   
00             83         9           8  ...         7           2       4   
01             73         9           8  ...         6           2       4   
02             77        10           8  ...         6           2       5   
03             93        13          13  ...         6           2       6   
04            105        13          14  ...         6           2       8   

year_term           2019.Fall                   2019.Spring                    
stage     Deposited  Accepted Applied Deposited    Accepted Applied Deposited  
00                3         2      19         1           2       8         2  
01                3         2      29         1           4      11         2  
02                3         2      37         1           5      13         3  
03                3         2      46         1           5      14         3  
04                3         2      56         1           6      14         3  

[5 rows x 48 columns]

In [21]:
summ_t.info()

<class 'pandas.core.frame.DataFrame'>
Index: 54 entries, 00 to 53
Data columns (total 48 columns):
(2012.Fall, Accepted)       54 non-null int64
(2012.Fall, Applied)        54 non-null int64
(2012.Fall, Deposited)      54 non-null int64
(2012.Spring, Accepted)     54 non-null int64
(2012.Spring, Applied)      54 non-null int64
(2012.Spring, Deposited)    54 non-null int64
(2013.Fall, Accepted)       54 non-null int64
(2013.Fall, Applied)        54 non-null int64
(2013.Fall, Deposited)      54 non-null int64
(2013.Spring, Accepted)     54 non-null int64
(2013.Spring, Applied)      54 non-null int64
(2013.Spring, Deposited)    54 non-null int64
(2014.Fall, Accepted)       54 non-null int64
(2014.Fall, Applied)        54 non-null int64
(2014.Fall, Deposited)      54 non-null int64
(2014.Spring, Accepted)     54 non-null int64
(2014.Spring, Applied)      54 non-null int64
(2014.Spring, Deposited)    54 non-null int64
(2015.Fall, Accepted)       54 non-null int64
(2015.Fall, Applied)       

In [29]:
terms = ['2012.Fall','2013.Fall','2014.Fall','2015.Fall','2016.Fall','2017.Fall','2018.Fall','2019.Fall']
stages = [ 'Applied', 'Accepted', 'Deposited']
for st in stages:
    df = summ_t.loc[:, (terms, st)]
    

year_term,2012.Fall,2013.Fall,2014.Fall,2015.Fall,2016.Fall,2017.Fall,2018.Fall,2019.Fall
stage,Deposited,Deposited,Deposited,Deposited,Deposited,Deposited,Deposited,Deposited
00,3,9,4,2,1,10,7,1
01,3,9,4,2,1,10,6,1
02,3,10,4,2,1,11,6,1
03,3,13,4,2,1,11,6,1
04,3,13,4,2,1,11,6,1
05,3,13,4,2,1,11,6,1
06,3,14,4,2,1,11,6,1
07,4,16,4,4,1,11,6,1
08,4,17,4,4,1,11,6,1
